In [121]:


import torch as th
import torch.distributions as dist

import numpy as np

import networkx as nx

import pandas as pd

import matplotlib.pyplot as plt

class ProdGraph:
    def __init__(self, adjacency, distributions, initialization):
        self.adjacency = th.tensor(adjacency)
        self.distributions = distributions
        
        G = nx.DiGraph(adjacency)
        topological_order = th.tensor(list(nx.topological_sort(G)))
        self.topological_order = topological_order
        
        self.remaining_time = th.zeros(adjacency.shape[0])
        self.states = th.zeros((adjacency.shape[0], adjacency.shape[0]))
        self.output_buffer = th.zeros(adjacency.shape[0])
        self.input_buffer = th.zeros(adjacency.shape[0])
        self.n_nodes = adjacency.shape[0]                   
        
        
        # Initialize
        self.states = th.tensor(initialization)
        
        for i in range(self.n_nodes):
            self.remaining_time[i] = distributions[i].sample()
            if th.sum(self.adjacency[:, i]) == 0:
                self.input_buffer[i] = th.inf
                
        self.lapsed_time = 0
        
        self.log = pd.DataFrame({'time': [], 'node': [], 'state': [], 'input_buffer': [], 'output_buffer': []})
        
    def forward(self):
        
        # Define entry point
        remaining_time = th.where(self.states[:, 0] * self.remaining_time > 0, self.remaining_time, float('inf'))
        min_value, entry_index = th.min(remaining_time, dim=0)
        
        # Adjust times
        lapsed_time = min_value.item()
        self.remaining_time -= lapsed_time * self.states[:, 0]
        self.lapsed_time += lapsed_time
        
        # Get current topological order
        topological_index_entry_node = th.where(self.topological_order == entry_index)[0].item()
        current_topological_order = [self.topological_order[(topological_index_entry_node + i) % self.n_nodes] for i in range(self.n_nodes)]
        
        # Iterate through graph
        for node_index in current_topological_order:
            
            # Move supplies output to input
            supply_vec = self.adjacency[:, node_index]
            if th.sum(supply_vec) > 0:
                num_supply_sets = th.min(th.nan_to_num(self.output_buffer / supply_vec, nan=float('inf')))
                self.output_buffer -= num_supply_sets * supply_vec
                self.input_buffer[node_index] = num_supply_sets
                
            # Case producing
            if self.states[node_index, 0] == 1:
                # Check if this node 
                if self.remaining_time[node_index] == 0:
                    # Check if the part is finished
                    self.output_buffer[node_index] += 1
                    
                    if self.input_buffer[node_index] > 0:
                        # Start next part
                        self.remaining_time[node_index] = self.distributions[node_index].sample()
                        self.input_buffer[node_index] -= 1
                        self.log = pd.concat([self.log, pd.DataFrame([{'time': self.lapsed_time, 'node': node_index.item(), 'input_buffer': self.input_buffer[node_index].item(), 'output_buffer': self.output_buffer[node_index].item(), 'state': 0}])])
                    else:
                        # Switch to starved
                        self.states[node_index, 0] = 0
                        self.states[node_index, 1] = 1
                        self.log = pd.concat([self.log, pd.DataFrame([{'time': self.lapsed_time, 'node': node_index.item(), 'input_buffer': self.input_buffer[node_index].item(), 'output_buffer': self.output_buffer[node_index].item(), 'state': 1}])])
                
            # Case Starved 
            elif self.states[node_index, 1] == 1:
                # Check if supplies are now here
                if self.input_buffer[node_index] > 0:
                    self.input_buffer[node_index] -= 1
                    self.remaining_time[node_index] = self.distributions[node_index].sample()
                    
                    # Switch to producing
                    self.states[node_index, 0] = 1
                    self.states[node_index, 1] = 0
                    self.log = pd.concat([self.log, pd.DataFrame([{'time': self.lapsed_time, 'node': node_index.item(), 'input_buffer': self.input_buffer[node_index].item(), 'output_buffer': self.output_buffer[node_index].item(), 'state': 0}])])
                
            # Case Blocked 
            elif self.states[node_index, 1] == 2:
                pass
                
            # Case Failure
            else:
                pass


In [124]:
adjacency_matrix = np.array([[0, 0, 1, 0],
                             [0, 0, 1, 0],
                             [0, 0, 0, 1],
                             [0, 0, 0, 0]])

init_matrix = np.array([[1, 0, 0, 0],
                             [1, 0, 0, 0],
                             [0, 1, 0, 0],
                             [0, 1, 0, 0]])

dists = [dist.Uniform(9, 11), dist.Uniform(9, 11), dist.Uniform(9, 11), dist.Uniform(9, 11)]

graph = ProdGraph(adjacency_matrix, dists, init_matrix)

for i in range(100):
    graph.forward()
    
graph.log

,time,node,state,input_buffer,output_buffer
0,10.115123,0.0,0.0,inf,1.0
0,10.115123,1.0,0.0,0.0,0.0
0,19.307267,0.0,0.0,inf,1.0
0,20.725791,1.0,1.0,0.0,1.0
0,20.725791,2.0,0.0,0.0,0.0
...,...,...,...,...,...
0,371.610536,0.0,0.0,inf,1.0
0,371.610536,1.0,0.0,0.0,0.0
0,372.075235,2.0,1.0,0.0,1.0
0,372.075235,3.0,0.0,0.0,19.0


In [64]:
import torch

# Input tensors
tensor1 = torch.tensor([1.0, 0.0, 2.0])
tensor2 = torch.tensor([1.0, 0.0, 1.0])
result1 = tensor1 / tensor2

tensor3 = torch.tensor([2.0, 2.0])
tensor4 = torch.tensor([1.0, 1.0])
result2 = tensor3 / tensor4

tensor5 = torch.tensor([3.0, 3.0, 0.0])
tensor6 = torch.tensor([1.0, 1.0, 1.0])
result3 = tensor5 / tensor6

# Print results
print("Result 1:", result1)
print("Result 2:", result2)
print("Result 3:", result3)

Result 1: tensor([1., nan, 2.])
Result 2: tensor([2., 2.])
Result 3: tensor([3., 3., 0.])
